In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals
from __future__ import print_function

In [4]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from make_pairwise_gold_metric_scores import compute_metrics

pd.set_option('max_colwidth',5000)

In [5]:
# Load word2vec model for this specific course
w2v_matrix = pickle.load( open( "../data/w2v_matrix_for_HarvardX__HDS_3221_2X__1T2016.p", "rb" ) )
vocab = np.array(pickle.load( open( "../data/vocab_for_HarvardX__HDS_3221_2X__1T2016.p", "rb" ) ) )

In [14]:
def tfidf_pca(gold_data, gold_matrix, eigenvector=20):
  # Get tfidf counts for each comment as a matrix C with shape (# comments, size of vocab)
  tfidf = TfidfVectorizer(stop_words='english')
  C = tfidf.fit_transform(gold_data.body.values)
  print("tfidf C shape", C.shape)
  pca = PCA(n_components=eigenvector)
  A = pca.fit_transform(C.toarray())
  print("PCA shape", A.shape)
  A=normalize(A,norm="l2")
  # We compute pairwise cosine similarity with dot product since A is normalized.
  pairwise_cosine_similarity = np.dot(A, A.transpose())
  return MinMaxScaler().fit_transform(pairwise_cosine_similarity)

In [15]:
# Load gold train data
gold_matrix_train = pd.read_csv('gold_matrix_train_HarvardX__HDS_3221_2X__1T2016.csv.gz', compression='gzip')
df_gold_train = pd.read_csv('gold_data_train_HarvardX__HDS_3221_2X__1T2016.csv.gz', compression='gzip')
pairwise_cosine_similarity_train = tfidf_pca(df_gold_train, gold_matrix_train)

tfidf C shape (761, 6942)
PCA shape (761, 20)


In [16]:
# Load gold data
gold_matrix_test = pd.read_csv('gold_matrix_test_HarvardX__HDS_3221_2X__1T2016.csv.gz', compression='gzip')
df_gold_test = pd.read_csv('gold_data_test_HarvardX__HDS_3221_2X__1T2016.csv.gz', compression='gzip')
pairwise_cosine_similarity_test = tfidf_pca(df_gold_test, gold_matrix_test)

tfidf C shape (649, 6583)
PCA shape (649, 20)


In [17]:
metrics = compute_metrics(pairwise_cosine_similarity_train, pairwise_cosine_similarity_test, gold_matrix_train, df_gold_train, gold_matrix_test, df_gold_test)
pretty_metrics = pd.DataFrame(pd.Series(metrics), columns = ["Score"])
pretty_metrics

Median Quantile (Rank) Difference score: 0.86371589811 - 0.429814981446 = 0.433900916664
Pairwise Binary Logistic Regression Accuracy score: 0.866738208124

The next test uses parameter optimization over a random forest
classifier's parameters and may take 30s to 2 min to run.

Pairwise Binary Random Forest Accuracy score: 0.866591009993


,Score
logreg_acc_pairwise_binary,0.866738
median_quantile_diff,0.433901
random_forest_acc_pairwise_binary,0.866591


In [18]:
# Switching train and test

metrics = compute_metrics(pairwise_cosine_similarity_test, pairwise_cosine_similarity_train, gold_matrix_test, df_gold_test, gold_matrix_train, df_gold_train)
pretty_metrics = pd.DataFrame(pd.Series(metrics), columns = ["Score"])
pretty_metrics

Median Quantile (Rank) Difference score: 0.83777138111 - 0.414315833824 = 0.423455547286
Pairwise Binary Logistic Regression Accuracy score: 0.84033043181

The next test uses parameter optimization over a random forest
classifier's parameters and may take 30s to 2 min to run.

Pairwise Binary Random Forest Accuracy score: 0.840352879623


,Score
logreg_acc_pairwise_binary,0.840330
median_quantile_diff,0.423456
random_forest_acc_pairwise_binary,0.840353
